In [7]:
import oda_api.token 
import logging
import numpy as np
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage, OdaLightCurve, OdaSpectrum
import matplotlib.pyplot as plt
import astroquery.heasarc
from astropy.wcs import WCS
from astropy.io import fits
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.time import Time
from matplotlib.patches import Circle
from astroquery.jplhorizons import Horizons
import pandas as pd
import astropy.units as u
import json
from collections import defaultdict
from astropy.io import ascii
import os

In [8]:
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Load the ScWs

In [9]:
jupiter_table = ascii.read('../data/jupiter_table.dat')

scw_ids = jupiter_table['scw_id'].data
scw_versions = jupiter_table['scw_ver'].data
scw = [str(id).zfill(12) + ".00" + str(ver) for id, ver in zip(scw_ids, scw_versions)]

start = jupiter_table['start_date'].data
end = jupiter_table['end_date'].data
durations = [(e - s) for e,s in zip(end, start)]

jupiter_ra = jupiter_table['jupiter_ra'].data
jupiter_dec = jupiter_table['jupiter_dec'].data

## QUERYING 

## IMAGES

In [4]:
isot_start_times = Time(start, format='mjd').isot
isot_end_times = Time(end, format='mjd').isot
duration_seconds = [duration * 86400 for duration in durations]
year_months = [st[:7] for st in isot_start_times]

15 - 30 keV

In [5]:
# Already processed ScWs

scw_to_match1 = []
directory1 = "../data/Jupiter/15-30keV/Images"
for filename in os.listdir(directory1):
    if filename.endswith(".fits"):
        scw_to_match1.append(filename[:16])

# scws different from the ones in the table
not_in_table = [s for s in scw_to_match1 if s not in scw]
print("Number of ScWs not in the table:", len(not_in_table))
print("Number of ScWs to query:", len(scw) - (len(scw_to_match1) - len(not_in_table)))

Number of ScWs not in the table: 0
Number of ScWs to query: 785


In [ ]:
disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for scw_id, start_time, end_time, jra, jdec in filtered_scws:
        
        # Check ScW isn't already in the FITS folder
        if scw_id in scw_to_match1:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "15",
            "E2_keV": "30",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

Trying SCW 026700060010.001
Is complete  False
Trying SCW 026700070010.001
Is complete  False
Trying SCW 026700080010.001
Is complete  False
Trying SCW 026700090010.001
Is complete  False
Trying SCW 026700100010.001
Is complete  False
Trying SCW 026700120010.001
Is complete  False
Trying SCW 026700130010.001
Is complete  False
Trying SCW 026700140010.001
Is complete  False
Trying SCW 026700150010.001
Is complete  False
Trying SCW 026700160010.001
Is complete  False
Trying SCW 026700170010.001
Is complete  False
Trying SCW 026700180010.001
Is complete  False
Trying SCW 026700190010.001
Is complete  False
Trying SCW 026700200010.001
Is complete  False
Trying SCW 026700210010.001
Is complete  False
Trying SCW 026700230010.001
Is complete  False
Trying SCW 026700240010.001
Is complete  False
Trying SCW 026700250010.001
Is complete  False
Trying SCW 026700260010.001
Is complete  False
Trying SCW 026700270010.001
Is complete  True
Trying SCW 026700280010.001
Is complete  False
Trying SCW 026

KeyboardInterrupt: 

In [ ]:
scw_to_match2 = []
directory2 = "../data/Jupiter/30-60keV/Images"
for filename in os.listdir(directory2):
    if filename.endswith(".fits"):
        scw_to_match2.append(filename[:16])

30 - 60 keV

In [ ]:
from collections import defaultdict

disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for scw_id, start_time, end_time, jra, jdec in filtered_scws:

        if scw_id in scw_to_match2:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "30",
            "E2_keV": "60",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

3 - 15 keV

In [10]:
scw_to_match3 = []
directory3 = "../data/Jupiter/3-15keV/Images"
for filename in os.listdir(directory3):
    if filename.endswith(".fits"):
        scw_to_match3.append(filename[:16])

In [11]:
jemx_validity = []
for s in scw:
    for row in jupiter_table:
        if str(row['scw_id']).zfill(12) == s[:12]:
            jemx_validity.append(row['jemx_valid'])

In [12]:
from collections import defaultdict

disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for i, (scw_id, start_time, end_time, jra, jdec) in enumerate(filtered_scws):

        if scw_id in scw_to_match3:
            continue

        if jemx_validity[i] == 0:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "3",
            "E2_keV": "15",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "jemx",
            "osa_version": "OSA11.2",
            "product": "jemx_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    # Summary of completed SCWs
    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

Trying SCW 026700060010.001
Is complete  False
Trying SCW 026700070010.001
Is complete  False
Trying SCW 026700080010.001
Is complete  False
Trying SCW 026700090010.001
Is complete  True
Trying SCW 026700100010.001
Is complete  False
Trying SCW 026700140010.001
Is complete  True
Trying SCW 026700150010.001
Is complete  False
Trying SCW 026700370010.001


KeyboardInterrupt: 

Save FITS

In [8]:
new_results = []
new_scws = []
for scw_id, data in data_by_date.items():
    if data is not None:
        new_results.append(data)
        new_scws.append(scw_id)

In [9]:
# Save FITS files
import os

output_dir = "../data/Jupiter/15-30keV/Images/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Check if the directory is empty
#if os.path.exists(output_dir) and os.listdir(output_dir):
   # print(f"Directory '{output_dir}' is not empty. Aborting to prevent overwriting.")
#else:
for i, data in enumerate(new_results):
    #if new_scws[i] in scw_to_match:
    im = OdaImage(data)
    im.write_fits(os.path.join(output_dir, f"{new_scws[i]}"))


## LIGHT CURVE

In [ ]:
api_cat={
    "cat_frame": "fk5",
    "cat_coord_units": "deg",
    "cat_column_list": [
        [0],
        ["Jupiter"],
        [125.4826889038086],
        [0], # this will be updated during the querying
        [0],
        [-32768],
        [2],
        [0],
        [0.0002800000074785203]],
    "cat_column_names": [
        "meta_ID",
        "src_names",
        "significance",
        "ra",
        "dec",
        "NEW_SOURCE",
        "ISGRI_FLAG",
        "FLAG",
        "ERR_RAD"
    ],
    "cat_column_descr":
        [
            ["meta_ID", "<i8"],
            ["src_names", "<U11"],
            ["significance", "<f8"],
            ["ra", "<f8"],
            ["dec", "<f8"],
            ["NEW_SOURCE", "<i8"],
            ["ISGRI_FLAG", "<i8"],
            ["FLAG", "<i8"],
            ["ERR_RAD", "<f8"]
        ],
    "cat_lat_name": "dec",
    "cat_lon_name": "ra"
}

In [ ]:
from collections import defaultdict

lc_disp_by_date = {}
lc_data_by_date = {}
successful_lc_scws = []
lc_results = []

filtered_scws = [
    (scw[i], isot_start_times[i], isot_end_times[i], duration_seconds[i], jupiter_ra[i], jupiter_dec[i])
    for i in range(len(scw))
]

rrrr_max_pppp = defaultdict(int)
for scw_id, *_ in filtered_scws:
    rrrr = scw_id[:4]
    pppp = int(scw_id[4:8])
    rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

filtered_scws = [
    (scw_id, start_time, end_time, dur, jra, jdec)
    for scw_id, start_time, end_time, dur, jra, jdec in filtered_scws
    if 5 < int(scw_id[4:8]) < rrrr_max_pppp[scw_id[:4]] - 10
]

if not filtered_scws:
    print("No SCWs found after filtering.")
else:
    print(f"{len(filtered_scws)} SCWs found after filtering.")

# Process each SCW directly in a single loop
for scw_id, start_time, end_time, duration, ra, dec in filtered_scws:
    print(f"Trying SCW {scw_id}")

    api_cat.update({
        "cat_column_list": [
            [0],
            ["Jupiter"],
            [125.4826889038086],
            [ra],  # Update RA
            [dec],  # Update DEC
            [-32768],
            [2],
            [0],
            [0.0002800000074785203]
        ]
    })

    par_dict = {
        "RA": ra,
        "DEC": dec,
        "E1_keV": "15",
        "E2_keV": "30", 
        "T_format": "isot",
        'T1': start_time,
        'T2': end_time,
        "time_bin": duration, 
        "instrument": "isgri",
        "osa_version": "OSA11.2",
        "product": "isgri_lc",
        "product_type": "Real",
        "scw_list": [scw_id],
        #'token': disp.disable_email_token(oda_api.token.discover_token()),
        'selected_catalog': json.dumps(api_cat)
        }
    
    if scw_id not in lc_disp_by_date:
        lc_disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

    _disp = lc_disp_by_date[scw_id]
    data = lc_data_by_date.get(scw_id, None)

    if data is None and not _disp.is_failed:
        try:
            if not _disp.is_submitted:
                data = _disp.get_product(**par_dict, silent=True)
            else:
                _disp.poll()

            print("Is complete ", _disp.is_complete)
            if _disp.is_complete:
                data = _disp.get_product(**par_dict, silent=True)
                lc_data_by_date[scw_id] = data

        except Exception as e:
            print(f"Query failed for SCW {scw_id}: {e}")
            continue

    successful_lc_scws.append(scw_id)
    lc_results.append(data)

# Summary of completed SCWs
n_complete = sum(1 for _disp in lc_disp_by_date.values() if _disp.is_complete)
print(f"complete {n_complete} / {len(lc_disp_by_date)}")

if n_complete == len(lc_disp_by_date):
    print("done!")
else:
    print("not done")

Save FITS

In [ ]:
# Save FITS files
for i, data in enumerate(lc_results):
    lc = OdaLightCurve(data)
    lc.write_fits(f"../data/JupiterLC_FITS_15_30/{successful_lc_scws[i]}")